In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # for measuring model
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

import random
from collections import Counter

In [ ]:
# import data
https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [1]:
df = pd.read_df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv", sep = ",")

NameError: name 'pd' is not defined